In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playernextngames, playergamelog

Implement Data Collection and Data Preprocessing Functions

In [ ]:
# Get player ID from name
def getPlayerID(name):
    player = players.find_players_by_full_name(name)
    return player[0]['id']

# Get team ID from name
def getTeamID(team):
    team = teams.find_team_by_abbreviation(team)
    return team[0]['id']

# Get next 5 games by player ID
def getNextGames(id):
    next = playernextngames.PlayerNextNGames(
        number_of_games="5",
        player_id=str(id),
        season_all="2023",
        season_type_all_star="Regular Season"
    )

    next_games = next.get_data_frames()[0]

    columns_to_loop = ['HOME_TEAM_ABBREVIATION', 'VISITOR_TEAM_ABBREVIATION']

    # Find player team each game
    teams = defaultdict(int)  # Initialize with int type
    for home, away in zip(next_games[columns_to_loop[0]], next_games[columns_to_loop[1]]):
        teams[home] += 1
        teams[away] += 1

    for team, amount in teams.items():
        if amount == 5:
            # Add column for player team
            next_games['PLAYER_TEAM_ABBREVIATION'] = team

    # Find opponent each game
    def findOpponent(row):
        if row['HOME_TEAM_ABBREVIATION'] == row['PLAYER_TEAM_ABBREVIATION']:
            return row['VISITOR_TEAM_ABBREVIATION']
        else:
            return row['HOME_TEAM_ABBREVIATION']
        
    # Add column for opponent
    next_games['OPPONENT_TEAM_ABBREVIATION'] = next_games.apply(findOpponent, axis=1)

    return next_games

# Get player season game log by ID
def getGameLog(id):
    stats = playergamelog.PlayerGameLog(
        player_id=str(id),
        season="2023",
        season_type_all_star="Regular Season",
        #date_from_nullable="MM/DD/YYYY"
    )

    game_log = stats.get_data_frames()[0]
    return game_log

id = getPlayerID("Lebron James")
getNextGames(id)

Implement Feature Selection

Implement Model Selection & Model Training

Implement Model Evaluation and Prediction

Graphing